# <center> AWS-foryou </center>
### <center> examples </center>
---

### Example 1
Running sklearndiabetes.py as the user's algorithm.

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import time

os.chdir("..")

from awsforyou import algo_runner
import examples.sklearn_diabetes as sdb

In [2]:
x, y = sdb.get_diabetes(0)

In [3]:
data_loc = './examples/x_diabetes.csv'
target_loc = './examples/y_diabetes.csv'

In [4]:
start = time.time()
best_model = sdb.run_sklearn_diabetes(data_loc, target_loc)
finish = time.time()
runtime = finish - start
print("runtime is %f seconds" % runtime)
print("best model is \n %s" % best_model)

linear regression score = 0.247268
best hyperparameters estimate from grid search = 
 SVR(C=20, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.237037
begining 6-components PCA decomposition
percentage of variance explained = 0.780063
repeat grid search with PCA-transformed data
best hyperparameters estimate from grid search = 
 SVR(C=40, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.229473
runtime is 107.314522 seconds
best model is 
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)


In [6]:
awsforyou.algo_runner.run_algo("run_sklearn_diabetes(data_loc='./examples/x_diabetes.csv', target_loc='./examples/y_diabetes.csv')", 'sklearn_diabetes', 3, 3)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

AttributeError: module 'awsforyou' has no attribute 'benchmark_runner'